## AMC Movies list

In [ ]:
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def clean_movie_list(processed_texts):
    cleaned_list = []
    for movie in processed_texts:
        cleaned_list.append(movie.replace('--', '-'))
    return cleaned_list

driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.amccinemas.com/movies")

wait = WebDriverWait(driver, 10)

# Wait for movie names to load
wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#Featured_now > ul > li:nth-child(1) > section > p.movie-ttl')))

# Scraping movie names
elements = driver.find_elements(By.CSS_SELECTOR, '#Featured_now > ul > li > section > p.movie-ttl')
processed_texts = []
for element in elements:
    text = element.text.lower()
    text = re.sub(r'[()?><!"<:]', '', text)
    text = text.replace(' ', '-')
    if text.endswith('english'):
        text += '-'
    processed_texts.append(text)

# Clean the list
processed_texts = clean_movie_list(processed_texts)

# Output the results
print(processed_texts)

# Save the list to a text file
with open('movie_list_amc.txt', 'w') as file:
    for item in processed_texts:
        file.write('%s\n' % item)

driver.quit()

## AMC Scraping code

In [ ]:
import time
import re
import os
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime

today = datetime.today()
td = today.strftime('%m-%d')

def clean_movie_list(processed_texts):
    cleaned_list = []
    for movie in processed_texts:
        cleaned_list.append(movie.replace('--', '-'))
    return cleaned_list

# Initialize the Chrome WebDriver
driver = webdriver.Chrome()

# Navigate to the website
link = 'https://www.amccinemas.com/movies'
driver.get(link)

# Extract movie titles
movies = driver.find_elements(By.XPATH, '//section[contains(@onclick, "Get Tickets click-Movieslist")]//p[@class="movie-ttl"]')
time.sleep(8)

# Process each extracted text
processed_texts = []
for element in movies:
    text = element.text.lower()  # Convert to lowercase
    text = re.sub(r'[()?><!"<:]', '', text)  # Remove special characters
    text = text.replace(' ', '-')
    if text.endswith('arabic'):
        text += '-'
    processed_texts.append(text)

# Clean the movie list
processed_texts = clean_movie_list(processed_texts)
processed_texts = [text.replace("alla3eb-ma3-el3yal", "alla3eb-ma3-el3yal-") for text in processed_texts]

# Output the results
print(processed_texts)

# Save the list to a text file
with open('teest.txt', 'w') as file:
    for item in processed_texts:
        file.write('%s\n' % item)

# Create a directory for screenshots if it doesn't exist
if not os.path.exists(f'amc_screenshots/{td}'):
    os.makedirs(f'amc_screenshots/{td}')

# Create a CSV file to save movie titles and show times
csv_file_path = 'amc_movie_showtimes.csv'
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Movie Title', 'Show Time'])

    # Visit each movie's page and take screenshots
    for index, movie_title in enumerate(processed_texts):
        movie_link = f'https://www.amccinemas.com/movies/{movie_title}'
        driver.get(movie_link)
        time.sleep(5)

        # Select the location
        rio = driver.find_element(By.XPATH, '//aside[@id="loadlocation"]')
        rio.click()
        time.sleep(2)
        driver.find_element(By.XPATH, '//a[@class="btn-clearall"]').click()
        time.sleep(2)
        rio.click()
        time.sleep(2)
        driver.find_element(By.XPATH,'//*[@id="loadlocation"]/div/div/div/ul/li[1]').click()
        time.sleep(2)
        driver.find_element(By.XPATH,'//*[@id="loadlocation"]/div/div/div/ul/li[5]').click()
        time.sleep(2)
        driver.find_element(By.XPATH,'//*[@id="loadlocation"]/div/div/div/ul/li[6]').click()
        time.sleep(2)
        driver.find_element(By.XPATH,'//*[@id="loadlocation"]/div/div/div/ul/li[7]').click()
        time.sleep(2)
        driver.find_element(By.XPATH,'//a[@class="btn-apply"]').click()
        time.sleep(5)
        
        try:
            # Find all movie times and click each one
            movie_times = driver.find_elements(By.XPATH, '//span[@class="amc-time "]')
            time.sleep(5)
            for time_index in range(len(movie_times)):
                # Re-locate movie times after each interaction
                movie_times = driver.find_elements(By.XPATH, '//span[@class="amc-time "]')
                time.sleep(5)
                show_time_text = movie_times[time_index].text  # Extract show time text
                movie_times[time_index].click()
                time.sleep(5)
                driver.find_element(By.XPATH, '//*[@id="gstid"]/a').click()
                driver.find_element(By.XPATH, '//*[@id="rulesofentry"]/section/section/section/section[2]/section/aside/a[1]').click()

                # Scroll down and zoom out for better screenshot visibility
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(4)
                driver.execute_script("document.body.style.zoom='70%'")
                time.sleep(5)

                # Take a screenshot
                screenshot_filename = f'amc_screenshot_{movie_title}_{time_index}_{td}.png'
                screenshot_path = os.path.join(f'amc_screenshots/{td}', screenshot_filename)
                
                try:
                    driver.save_screenshot(screenshot_path)
                    print(f"Screenshot saved: {screenshot_path}")
                except Exception as e:
                    print(f"Failed to save screenshot for {movie_title} at time index {time_index}: {e}")

                time.sleep(4)

                # Save movie title and show time to the CSV file
                csv_writer.writerow([movie_title.replace('-', ' ').title(), show_time_text])

                # Navigate back to the movie's page
                driver.get(movie_link)
                time.sleep(4)
        except Exception as e:
            print(f"Error with movie {movie_title}: {e}")

# Quit the driver
driver.quit()


## FUNCTION FOR CHECK ALL SEATS ( MODAY )

In [ ]:
import cv2
import os
import csv

def process_seat_images(directory):
    # Get the list of image files in the directory
    image_files = os.listdir(directory)

    # Open a CSV file to write the results
    with open('seat_counts_unique_MONDAY.csv', mode='w', newline='') as csv_file:
        fieldnames = ['Movie Name', 'Available Seats', 'Unavailable Seats', 'Percent Available', 'Percent Unavailable']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()

        seen_movies = set()

        # Iterate over each image file
        for image_file in image_files:
            # Construct the full path to the image file
            image_path = os.path.join(directory, image_file)
            
            # Extract the movie name from the file name
            parts = image_file.split('_')
            movie_name = parts[2]

            # Skip if the movie name has already been processed
            if movie_name in seen_movies:
                continue

            seen_movies.add(movie_name)

            # Load the image
            img = cv2.imread(image_path)
            
            # Crop image
            img = img[100:-200,]

            # Convert the image to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Apply Gaussian blur to reduce noise
            blurred = cv2.GaussianBlur(gray, (5, 5), 0)

            # Perform Canny edge detection
            edges = cv2.Canny(blurred, 50, 150)

            # Find contours in the edges image
            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Define area thresholds for available and unavailable seats
            min_area_available = 530
            max_area_available = 560

            min_area_unavailable = 466
            max_area_unavailable = 520

            # Initialize counters for available and unavailable seats
            available_seats = 0
            unavailable_seats = 0

            # Iterate over each contour
            for contour in contours:
                # Calculate the area of the contour
                area = cv2.contourArea(contour)

                # Check if the area is within the desired range for available seats
                if min_area_available < area < max_area_available:
                    # Increment the count of available seats
                    available_seats += 1
                    # Draw the contour (optional)
                    cv2.drawContours(img, [contour], -1, (0, 255, 0), 2)

                # Check if the area is within the desired range for unavailable seats
                elif min_area_unavailable < area < max_area_unavailable:
                    # Increment the count of unavailable seats
                    unavailable_seats += 1
                    # Draw the contour (optional)
                    cv2.drawContours(img, [contour], -1, (0, 0, 255), 2)

            # Calculate the percentages of available and unavailable seats
            total_seats = available_seats + unavailable_seats
            if total_seats > 0:
                percent_available = (available_seats / total_seats) * 100
                percent_unavailable = (unavailable_seats / total_seats) * 100
            else:
                percent_available = 0
                percent_unavailable = 0

            # Write the results to the CSV file
            writer.writerow({
                'Movie Name': movie_name,
                'Available Seats': available_seats,
                'Unavailable Seats': unavailable_seats,
                'Percent Available': percent_available,
                'Percent Unavailable': percent_unavailable
            })

            # Print the counts of available and unavailable seats for each image
            print("Image:", image_file)
            print("Available Seats:", available_seats)
            print("Unavailable Seats:", unavailable_seats)

            # Show the result with contours (optional)
            # cv2.imshow("Result", img)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()

# Example usage
directory = 'amc_screenshots_MONDAY'
process_seat_images(directory)


## FUNCTION FOR CHECK ALL SEATS ( WEDNESDAY )

In [ ]:
import cv2
import os
import csv

def process_seat_images(directory):
    # Get the list of image files in the directory
    image_files = os.listdir(directory)

    # Open a CSV file to write the results
    with open('seat_counts_unique_WEDNESDAY.csv', mode='w', newline='') as csv_file:
        fieldnames = ['Movie Name', 'Available Seats', 'Unavailable Seats', 'Percent Available', 'Percent Unavailable']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()

        seen_movies = set()

        # Iterate over each image file
        for image_file in image_files:
            # Construct the full path to the image file
            image_path = os.path.join(directory, image_file)
            
            # Extract the movie name from the file name
            parts = image_file.split('_')
            movie_name = parts[2]

            # Skip if the movie name has already been processed
            if movie_name in seen_movies:
                continue

            seen_movies.add(movie_name)

            # Load the image
            img = cv2.imread(image_path)
            
            # Crop image
            img = img[100:-200,]

            # Convert the image to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Apply Gaussian blur to reduce noise
            blurred = cv2.GaussianBlur(gray, (5, 5), 0)

            # Perform Canny edge detection
            edges = cv2.Canny(blurred, 50, 150)

            # Find contours in the edges image
            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Define area thresholds for available and unavailable seats
            min_area_available = 730
            max_area_available = 900

            min_area_unavailable = 500
            max_area_unavailable = 890

            # Initialize counters for available and unavailable seats
            available_seats = 0
            unavailable_seats = 0

            # Iterate over each contour
            for contour in contours:
                # Calculate the area of the contour
                area = cv2.contourArea(contour)

                # Check if the area is within the desired range for available seats
                if min_area_available < area < max_area_available:
                    # Increment the count of available seats
                    available_seats += 1
                    # Draw the contour (optional)
                    cv2.drawContours(img, [contour], -1, (0, 255, 0), 2)

                # Check if the area is within the desired range for unavailable seats
                elif min_area_unavailable < area < max_area_unavailable:
                    # Increment the count of unavailable seats
                    unavailable_seats += 1
                    # Draw the contour (optional)
                    cv2.drawContours(img, [contour], -1, (0, 0, 255), 2)

            # Calculate the percentages of available and unavailable seats
            total_seats = available_seats + unavailable_seats
            if total_seats > 0:
                percent_available = (available_seats / total_seats) * 100
                percent_unavailable = (unavailable_seats / total_seats) * 100
            else:
                percent_available = 0
                percent_unavailable = 0

            # Write the results to the CSV file
            writer.writerow({
                'Movie Name': movie_name,
                'Available Seats': available_seats,
                'Unavailable Seats': unavailable_seats,
                'Percent Available': percent_available,
                'Percent Unavailable': percent_unavailable
            })

            # Print the counts of available and unavailable seats for each image
            print("Image:", image_file)
            print("Available Seats:", available_seats)
            print("Unavailable Seats:", unavailable_seats)

            # Show the result with contours (optional)
            # cv2.imshow("Result", img)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()

# Example usage
directory = 'amc_screenshots_WEDNESDAY'
process_seat_images(directory)


## function to create csv with all movies name, show time and branch


In [ ]:
import time
import re
import os
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from selenium.common.exceptions import NoSuchElementException

def scrape_amc_movie_showtimes():
    today = datetime.today()
    td = today.strftime('%m-%d')

    def clean_movie_list(processed_texts):
        cleaned_list = []
        for movie in processed_texts:
            cleaned_list.append(movie.replace('--', '-'))
        return cleaned_list

    # Initialize the Chrome WebDriver
    driver = webdriver.Chrome()

    try:
        # Navigate to the website
        link = 'https://www.amccinemas.com/movies'
        driver.get(link)
        time.sleep(8)  # Wait for the page to load

        # Extract movie titles
        movies = driver.find_elements(By.XPATH, '//section[contains(@onclick, "Get Tickets click-Movieslist")]//p[@class="movie-ttl"]')

        # Process each extracted text
        processed_texts = []
        for element in movies:
            text = element.text.lower()  # Convert to lowercase
            text = re.sub(r'[()?><!"<:]', '', text)  # Remove special characters
            text = text.replace(' ', '-')
            if text.endswith('arabic'):
                text += '-'
            processed_texts.append(text)

        # Clean the movie list
        processed_texts = clean_movie_list(processed_texts)
        processed_texts = [text.replace("alla3eb-ma3-el3yal", "alla3eb-ma3-el3yal-") for text in processed_texts]

        # Output the results
        print(processed_texts)

        # Create a CSV file to save movie titles, show times, and branch names
        csv_file_path = 'amc_movie_showtimes.csv'
        with open(csv_file_path, 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(['Movie Title', 'Show Time', 'Branch Name'])

            # Flag to check if location selection has been performed
            location_selected = False

            # Visit each movie's page and gather information
            for index, movie_title in enumerate(processed_texts):
                movie_link = f'https://www.amccinemas.com/movies/{movie_title}'
                driver.get(movie_link)
                time.sleep(5)

                # Check for the location selection element
                try:
                    rio = driver.find_element(By.XPATH, '//aside[@id="loadlocation"]')
                    if not location_selected:
                        rio.click()
                        time.sleep(2)
                        driver.find_element(By.XPATH, '//a[@class="btn-clearall"]').click()
                        time.sleep(2)
                        rio.click()
                        time.sleep(2)
                        driver.find_element(By.XPATH, '//*[@id="loadlocation"]/div/div/div/ul/li[1]').click()
                        time.sleep(2)
                        driver.find_element(By.XPATH, '//*[@id="loadlocation"]/div/div/div/ul/li[5]').click()
                        time.sleep(2)
                        driver.find_element(By.XPATH, '//*[@id="loadlocation"]/div/div/div/ul/li[6]').click()
                        time.sleep(2)
                        driver.find_element(By.XPATH, '//*[@id="loadlocation"]/div/div/div/ul/li[7]').click()
                        time.sleep(2)
                        driver.find_element(By.XPATH, '//a[@class="btn-apply"]').click()
                        time.sleep(5)
                        location_selected = True

                except NoSuchElementException:
                    print(f"Location selection not found for {movie_title}. Skipping to the next movie.")
                    continue

                try:
                    # Find all movie times and branches
                    movie_times = driver.find_elements(By.XPATH, '//span[@class="amc-time "]')
                    branch_names = driver.find_elements(By.XPATH, '//h2[@data-moviename]')
                    time.sleep(5)
                    for time_index in range(len(movie_times)):
                        show_time_text = movie_times[time_index].text  # Extract show time text
                        branch_name_text = branch_names[time_index].text  # Extract branch name text

                        # Save movie title, show time, and branch name to the CSV file
                        csv_writer.writerow([movie_title.replace('-', ' ').title(), show_time_text, branch_name_text])
                        
                        time.sleep(1)
                except Exception as e:
                    print(f"Error with movie {movie_title}: {e}")
    finally:
        # Quit the driver
        driver.quit()

# Example usage
scrape_amc_movie_showtimes()


## loop Check 5 min

In [ ]:
import time
import re
import os
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta

def scrape_amc_movie_showtimes():
    today = datetime.today()
    td = today.strftime('%m-%d')

    def clean_movie_list(processed_texts):
        cleaned_list = []
        for movie in processed_texts:
            cleaned_list.append(movie.replace('--', '-'))
        return cleaned_list

    # Initialize the Chrome WebDriver
    driver = webdriver.Chrome()

    try:
        while True:  # Run indefinitely
            # Navigate to the website
            link = 'https://www.amccinemas.com/movies'
            driver.get(link)
            time.sleep(8)  # Wait for the page to load

            # Extract movie titles
            movies = driver.find_elements(By.XPATH, '//section[contains(@onclick, "Get Tickets click-Movieslist")]//p[@class="movie-ttl"]')

            # Process each extracted text
            processed_texts = []
            for element in movies:
                text = element.text.lower()  # Convert to lowercase
                text = re.sub(r'[()?><!"<:]', '', text)  # Remove special characters
                text = text.replace(' ', '-')
                if text.endswith('arabic'):
                    text += '-'
                processed_texts.append(text)

            # Clean the movie list
            processed_texts = clean_movie_list(processed_texts)
            processed_texts = [text.replace("alla3eb-ma3-el3yal", "alla3eb-ma3-el3yal-") for text in processed_texts]

            # Output the results
            print(processed_texts)

            # Save the list to a text file
            with open('movie_list_amc.txt', 'w') as file:
                for item in processed_texts:
                    file.write('%s\n' % item)

            # Create a directory for screenshots if it doesn't exist
            if not os.path.exists(f'FiveMinutes_screenshots/{td}'):
                os.makedirs(f'FiveMinutes_screenshots/{td}')

            # Create a CSV file to save movie titles and show times
            csv_file_path = 'FiveMinutes.csv'
            with open(csv_file_path, 'a', newline='') as csv_file:
                csv_writer = csv.writer(csv_file)
                csv_writer.writerow(['Movie Title', 'Show Time'])

                # Visit each movie's page and take screenshots
                for index, movie_title in enumerate(processed_texts):
                    movie_link = f'https://www.amccinemas.com/movies/{movie_title}'
                    driver.get(movie_link)
                    time.sleep(2)  # Reduce waiting time

                    # Select the location
                    try:
                        rio = driver.find_element(By.XPATH, '//aside[@id="loadlocation"]')
                        rio.click()
                        time.sleep(1)  # Reduce waiting time
                        driver.find_element(By.XPATH, '//a[@class="btn-clearall"]').click()
                        time.sleep(1)  # Reduce waiting time
                        rio.click()
                        time.sleep(1)  # Reduce waiting time
                        driver.find_element(By.XPATH, '//*[@id="loadlocation"]/div/div/div/ul/li[1]').click()
                        time.sleep(1)  # Reduce waiting time
                        driver.find_element(By.XPATH, '//*[@id="loadlocation"]/div/div/div/ul/li[5]').click()
                        time.sleep(1)  # Reduce waiting time
                        driver.find_element(By.XPATH, '//*[@id="loadlocation"]/div/div/div/ul/li[6]').click()
                        time.sleep(1)  # Reduce waiting time
                        driver.find_element(By.XPATH, '//*[@id="loadlocation"]/div/div/div/ul/li[7]').click()
                        time.sleep(1)  # Reduce waiting time
                        driver.find_element(By.XPATH, '//a[@class="btn-apply"]').click()
                        time.sleep(2)  # Reduce waiting time
                    except Exception as e:
                        print(f"Location selection error for {movie_title}: {e}")
                        continue

                    try:
                        # Find all movie times and check each one
                        movie_times = driver.find_elements(By.XPATH, '//span[@class="amc-time "]')
                        time.sleep(2)  # Reduce waiting time
                        movie_screenshot_taken = False  # Flag to check if screenshot was taken
                        for time_index in range(len(movie_times)):
                            # Re-locate movie times after each interaction
                            movie_times = driver.find_elements(By.XPATH, '//span[@class="amc-time "]')
                            time.sleep(1)  # Reduce waiting time
                            show_time_text = movie_times[time_index].text  # Extract show time text

                            # Parse showtime text to datetime object
                            show_time = datetime.strptime(show_time_text, '%I:%M %p')
                            show_time_file = show_time.strftime('%H:%M')
                            show_time_file = show_time_file.replace(':', '_')
                            now = datetime.now()
                            current_time_text = now.strftime('%I:%M %p')
                            current_time = datetime.strptime(current_time_text, '%I:%M %p')

                            # Replace the date of show_time and current_time with the same date
                            show_time = show_time.replace(year=now.year, month=now.month, day=now.day)
                            current_time = current_time.replace(year=now.year, month=now.month, day=now.day)

                            # Check if showtime is within 5 minutes from current time
                           # Calculate the time difference between showtime and current time
                            time_difference = show_time - current_time

                            # Check if the time difference is 5 minutes or less
                            if abs(time_difference) <= timedelta(minutes=5):
                                # Code to click on the showtime and take a screenshot
                                movie_times[time_index].click()
                                time.sleep(2)  # Reduce waiting time
                                driver.find_element(By.XPATH, '//*[@id="gstid"]/a').click()
                                driver.find_element(By.XPATH, '//*[@id="rulesofentry"]/section/section/section/section[2]/section/aside/a[1]').click()

                                # Scroll down and zoom out for better screenshot visibility
                                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                                time.sleep(1)  # Reduce waiting time
                                driver.execute_script("document.body.style.zoom='70%'")
                                time.sleep(1)  # Reduce waiting time

                                # Take a screenshot
                                screenshot_filename = f'amc_screenshot_{movie_title}_{td}_{show_time_file}.png'
                                screenshot_path = os.path.join(f'FiveMinutes_screenshots/{td}', screenshot_filename)
                                
                                try:
                                    driver.save_screenshot(screenshot_path)
                                    print(f"Screenshot saved: {screenshot_path}")
                                except Exception as e:
                                    print(f"Failed to save screenshot for {movie_title} at time index {time_index}: {e}")

                                time.sleep(1)  # Reduce waiting time

                                # Save movie title and show time to the CSV file
                                csv_writer.writerow([movie_title.replace('-', ' ').title(), show_time_text])
                                movie_screenshot_taken = True

                                # Navigate back to the movie's page
                                driver.get(movie_link)
                                time.sleep(2)  # Reduce waiting time
                                break  # Exit loop if screenshot was taken
 # Exit loop if screenshot was taken
                        if not movie_screenshot_taken:
                            print(f"No showtimes within 5 minutes for {movie_title}")
                    except Exception as e:
                        print(f"Error with movie {movie_title}: {e}")

            print("Sleeping for a short while before checking again...")
            time.sleep(30)  # Shorter wait before checking again
    finally:
        # Quit the driver
        driver.quit()

# Example usage
scrape_amc_movie_showtimes()


## MDBI Movies ratings

In [2]:
import imdb
import csv

def get_movie_details(movies):
    # Create an instance of the IMDb class
    ia = imdb.IMDb()
    
    details = {}
    
    for movie in set(movies):  # Using set to avoid duplicate searches
        try:
            # Search for the movie
            search_results = ia.search_movie(movie)
            
            if search_results:
                # Get the first movie from the search results
                movie_id = search_results[0].movieID
                movie_data = ia.get_movie(movie_id)
                
                # Extract the rating, genres, plot, director, actors, release date, runtime, and poster
                rating = movie_data.get('rating')
                genres = movie_data.get('genres')
                plot = movie_data.get('plot summary')
                directors = [director['name'] for director in movie_data.get('directors', [])]
                actors = [actor['name'] for actor in movie_data.get('cast', [])[:5]]  # Get top 5 actors
                release_date = movie_data.get('original air date')
                runtime = movie_data.get('runtimes')
                
                if plot:
                    plot = plot[0] if isinstance(plot, list) else plot
                
                details[movie] = {
                    'rating': rating if rating else 'Rating not found',
                    'genres': genres if genres else 'Genres not found',
                    'plot': plot if plot else 'Plot not found',
                    'directors': directors if directors else ['Director not found'],
                    'actors': actors if actors else ['Actors not found'],
                    'release_date': release_date if release_date else 'Release date not found',
                    'runtime': runtime[0] if runtime else 'Runtime not found'
                }
            else:
                details[movie] = {
                    'rating': 'Movie not found',
                    'genres': 'Movie not found',
                    'plot': 'Movie not found',
                    'directors': ['Movie not found'],
                    'actors': ['Movie not found'],
                    'release_date': 'Movie not found',
                    'runtime': 'Movie not found'
                }
        except Exception as e:
            details[movie] = {
                'rating': f'Error: {str(e)}',
                'genres': f'Error: {str(e)}',
                'plot': f'Error: {str(e)}',
                'directors': [f'Error: {str(e)}'],
                'actors': [f'Error: {str(e)}'],
                'release_date': f'Error: {str(e)}',
                'runtime': f'Error: {str(e)}'
            }
    
    # Assign details to duplicates
    final_details = {movie: details[movie] for movie in movies}
    return final_details

# Read movie names from movie_list.txt file
with open('movie_list_amc.txt', 'r') as file:
    movies_list = [line.strip() for line in file.readlines()]

details = get_movie_details(movies_list)

# Create a CSV file to output the movie details
with open('movie_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Movie Title", "Rating", "Genres", "Plot", "Directors", "Actors", "Release Date", "Runtime"])  # Header row
    
    for movie, detail in details.items():
        genres = ', '.join(detail['genres']) if isinstance(detail['genres'], list) else detail['genres']
        directors = ', '.join(detail['directors'])
        actors = ', '.join(detail['actors'])
        writer.writerow([movie, detail['rating'], genres, detail['plot'], directors, actors, detail['release_date'], detail['runtime']])